In [1]:
!pip3 install google-cloud-aiplatform==1.4.0

     |████████████████████████████████| 1.3 MB 7.1 MB/s eta 0:00:01
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.3.0
    Uninstalling google-cloud-aiplatform-1.3.0:
      Successfully uninstalled google-cloud-aiplatform-1.3.0


In [14]:
from google.cloud import aiplatform
PROJECT_ID=!gcloud config get-value project # returns default project id 
PROJECT_ID=PROJECT_ID[0]
REGION='europe-west4'
BUCKET = "gs://"+PROJECT_ID
DATA_BQ_URI = 'bq://'+PROJECT_ID+'.telco.churn'

In [15]:
VERSION='v1'
JOB_NAME="telco-churn-job-"+VERSION
MODEL_NAME="telco-churn-model-"+VERSION
DATASET_ID='5848592619210276864'

In [13]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET, location=REGION)

# ds = aiplatform.TabularDataset.create(display_name=display_name, gcs_source=gcs_source) # GCS dataset
ds = aiplatform.TabularDataset.create(display_name="churn-dataset-bq-"+VERSION, bq_source=DATA_BQ_URI) # BQ dataset

# ds.wait()


#ds = aiplatform.TabularDataset('5848592619210276864') # For existing dataset! TODO: Change that to the created dataset id

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=JOB_NAME, 
    python_package_gcs_uri=BUCKET+'/telco-churn/dist/trainer-0.1.tar.gz', 
    python_module_name='trainer.task', 
    container_uri='eu.gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest', 
    model_serving_container_image_uri='eu.gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest'
)

job.run(
    ds, replica_count=1, model_display_name=MODEL_NAME,
    base_output_dir=BUCKET+"/telco-churn/assets-"+VERSION,
    machine_type='n1-standard-4',
    bigquery_destination='bq://'+PROJECT_ID
)

m = job.get_model()

endpoint = m.deploy(machine_type='n1-standard-4')

endpoint.predict([['Male', 0, True, False, True, False, 8, 20.25, '158.35', 'No', 'No internet service', 'No internet service', 'No internet service', 'No internet service', 'No internet service', 'Month-to-month', 'Mailed check', 'No'], ['Female', 0, False, False, True, True, 2, 87.15, '183.75', 'Fiber optic', 'No', 'No', 'No', 'Yes', 'No', 'Month-to-month', 'Bank transfer (automatic)', 'No']])

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/478111835512/locations/europe-west4/datasets/8806771479136960512/operations/658455732433190912
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/478111835512/locations/europe-west4/datasets/8806771479136960512
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/478111835512/locations/europe-west4/datasets/8806771479136960512')
INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://myfirstproject-226013/telco-churn/assets-v1 
INFO:google.cloud.aiplatform.training_jobs:No dataset split provided. The service will use a default split.
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/location

RuntimeError: Training failed with:
code: 3
message: "The replica workerpool0-0 exited with a non-zero status of 1. \nTraceback (most recent call last):\n  File \"/usr/lib/python3.7/runpy.py\", line 193, in _run_module_as_main\n    \"__main__\", mod_spec)\n  File \"/usr/lib/python3.7/runpy.py\", line 85, in _run_code\n    exec(code, run_globals)\n  File \"/root/.local/lib/python3.7/site-packages/trainer/task.py\", line 185, in <module>\n    df_train = load_data_from_bq(arguments[\'training_data_uri\'])\nNameError: name \'load_data_from_bq\' is not defined\n\nTo find out more about why your job exited please check the logs: https://console.cloud.google.com/logs/viewer?project=478111835512&resource=ml_job%2Fjob_id%2F7328825541266505728&advancedFilter=resource.type%3D%22ml_job%22%0Aresource.labels.job_id%3D%227328825541266505728%22"
